<a href="https://colab.research.google.com/github/nicoaira/mabR-proteomics/blob/main/mabRi_proteomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1- Descargas e importaciones

In [ ]:
!pip install padua
import pandas as pd
import matplotlib.pyplot as plt
%load_ext rpy2.ipython
import numpy as np
import math
import padua
import scipy

2 - Descarga del dataset de datos proteomicos

In [2]:
!gdown 1uzjgUziJE26X61tQe4vV-NfAyZT8BXrL

Downloading...
From: https://drive.google.com/uc?id=1uzjgUziJE26X61tQe4vV-NfAyZT8BXrL
To: /content/data.txt
100% 5.43M/5.43M [00:00<00:00, 93.5MB/s]


3- Creacion del dataframe

In [3]:
df = pd.read_csv("data.txt", sep = '\t')

4- Renombrado de las muestras

In [4]:
renombrar = {'LFQ intensity Inductor 0 - 1' : 'Sin ATc - 1',
             'LFQ intensity Inductor 0 - 2' : 'Sin ATc - 2',
             'LFQ intensity Inductor 0 - 3' : 'Sin ATc - 3',
             'LFQ intensity Inductor 0 - 5' : 'Sin ATc - 5',
             'LFQ intensity Inductor 0.25 - 1' : '0.25 ATc - 1',
             'LFQ intensity Inductor 0.25 - 3' : '0.25 ATc - 3',
             'LFQ intensity Inductor 0.25 - 4' : '0.25 ATc - 4',
             'LFQ intensity Inductor 0.25 - 5' : '0.25 ATc - 5'
}

df.rename(columns = renombrar, inplace = True)

In [5]:
muestras = list(renombrar.values())

5- Se filtran las filas categorizadas como:


*   Only identified by site
*   Reverse
*   Potential contaminant

In [6]:
df = df[(df['Only identified by site'] != '+')]
df = df[(df['Reverse'] != '+')]
df = df[(df['Potential contaminant'] != '+')]

Se genera una copia del dataframe (df2) que se utilizará para un procesamiento en paralelo, sin la muestra Sin ATc - 2, la cual tuvo mayor variación

In [7]:
df2 = df.copy()

6- Se filtran las filas en los que en ninguno de los grupos hay mas de tres mediciones

In [8]:
df['conteo_0'] = (df[muestras[:4]] != 0 ).sum(axis=1)
df['conteo_0.25'] = (df[muestras[4:]] != 0 ).sum(axis=1)

df = df[(df['conteo_0'] >= 3) | (df['conteo_0.25'] >= 3)]

In [9]:
print('Cantidad de filas restantes:', df.shape[0])

Cantidad de filas restantes: 2191


7- Se cambian los 0 por NaNs

In [10]:
def fill_na(x):
  if x == 0:
    a = np.nan
  else:
    a = math.log(x, 2)
    return a

In [11]:
log_muestras = []
for muestra in muestras:
  nombre = 'log_' + muestra
  log_muestras.append(nombre)
  df[nombre] = df[muestra].apply(fill_na)

8- Para imputar los valores no leidos. Se genera una distribucion normal en cada columna y se samplean numeros aleatorios de la parte central de esa campana (el ancho de la campana usado es el argumento width). Despues se hace un dowshift de 1.8 unidades de desvio estandar. Esto se hace porque en general los valores que no fueron leidos es porque tienen baja abundancia, por lo que se asume que estarán en la cola izquierda de la distribución.

In [12]:
df[log_muestras] = padua.imputation.gaussian(df[log_muestras], width=0.3, downshift=-1.8, prefix=None)[0]

Una muestra de como queda formateado el dataframe

In [13]:
df[log_muestras].head(5)

,log_Sin ATc - 1,log_Sin ATc - 2,log_Sin ATc - 3,log_Sin ATc - 5,log_0.25 ATc - 1,log_0.25 ATc - 3,log_0.25 ATc - 4,log_0.25 ATc - 5
0,27.595838,28.067688,27.777748,27.854955,27.409489,27.409003,27.464353,27.300424
1,29.931287,30.263605,29.904692,30.127802,28.925356,29.666651,29.542354,29.794299
2,26.365246,26.567178,26.213693,26.468865,25.503822,25.885625,25.904284,25.964970
3,29.538455,30.131241,29.503606,29.578145,29.558774,29.478887,29.498327,28.890817
4,29.565327,30.468593,29.616265,29.670134,29.473285,29.586545,29.496614,28.570873


9- Se realiza un t-test en cada peptido para conocer el p-value entre las muestras con y sin ATc

In [14]:
df['pvalue'] = scipy.stats.ttest_ind(df[log_muestras[:4]], df[log_muestras[4:]], axis=1)[1]

10- Se computa el log10 del p-value para luego graficarlo

In [15]:
df['log_pvalue'] = df['pvalue'].apply(lambda x: math.log(x, 10))

In [16]:
df[['pvalue','log_pvalue']].head()

,pvalue,log_pvalue
0,0.006112,-2.213852
1,0.034194,-1.466051
2,0.003878,-2.411419
3,0.175435,-0.755884
4,0.137678,-0.861134


11- Las columnas de "muestras" que contienen los valores de las intensidades continenen ceros que deben rellenarse con los datos de imputación. Para esto, se cambian por los valores del log2 de las intensidades elevandolas al cuadrado. En los casos en que no hubo imputación debería devolver el mismo valor de intensidad original y en los casos en los que hubo imputación devuelve el nuevo valor imputado

In [17]:
df[muestras[:]].iloc[12]

Sin ATc - 1     20436000.0
Sin ATc - 2     27112000.0
Sin ATc - 3            0.0
Sin ATc - 5            0.0
0.25 ATc - 1           0.0
0.25 ATc - 3    20418000.0
0.25 ATc - 4    34781000.0
0.25 ATc - 5    15807000.0
Name: 14, dtype: float64

In [18]:
for muestra in muestras:
  nombre = 'log_' + muestra
  df[muestra] =  2 ** df[nombre]

In [19]:
df[muestras[:]].iloc[12]

Sin ATc - 1     2.043600e+07
Sin ATc - 2     2.711200e+07
Sin ATc - 3     1.633552e+07
Sin ATc - 5     7.871172e+06
0.25 ATc - 1    6.402082e+06
0.25 ATc - 3    2.041800e+07
0.25 ATc - 4    3.478100e+07
0.25 ATc - 5    1.580700e+07
Name: 14, dtype: float64

12- Calculo del fold change y el log2 del fold change en cada fila

In [20]:
df['fold_change'] = df[muestras[4:]].mean(axis=1)/df[muestras[:4]].mean(axis=1)

In [21]:
df['log_fc'] = df['fold_change'].apply(lambda x: math.log(x, 2))

In [22]:
df[['log_fc', 'log_pvalue']].head()

,log_fc,log_pvalue
0,-0.436986,-2.213852
1,-0.546506,-1.466051
2,-0.584018,-2.411419
3,-0.332002,-0.755884
4,-0.547500,-0.861134


13- Descarga del dataframe para mapear los codigos de uniprot de Mtb H37Ra con los locus de Mtb H37Rv

In [23]:
!gdown 18E0ns3U_6a2UIA2QNv6o8ExIWAkIQW3R

Downloading...
From: https://drive.google.com/uc?id=18E0ns3U_6a2UIA2QNv6o8ExIWAkIQW3R
To: /content/ra_rv_dataset.csv
100% 505k/505k [00:00<00:00, 114MB/s]


In [24]:
df_rv_ra = pd.read_csv('ra_rv_dataset.csv')

In [25]:
df_rv_ra.head()

,Index,Locus tag,Ra,Ra_UNIPROT,#Name,Accession,Start,Stop,Strand,GeneID,Locus,Protein product,Length,Protein Name
0,0,Rv0001,mra:MRA_0001,A5TY69,chromosome,NC_000962.3,1.0,1524.0,+,885041.0,dnaA,NP_214515.1,507.0,chromosomal replication initiator protein DnaA
1,1,Rv0002,mra:MRA_0002,A5TY70,chromosome,NC_000962.3,2052.0,3260.0,+,887092.0,dnaN,NP_214516.1,402.0,DNA polymerase III subunit beta
2,2,Rv0003,mra:MRA_0003,A5TY71,chromosome,NC_000962.3,3280.0,4437.0,+,887089.0,recF,NP_214517.1,385.0,DNA replication/repair protein RecF
3,3,Rv0004,mra:MRA_0004,A5TY72,chromosome,NC_000962.3,4434.0,4997.0,+,887088.0,NaN,NP_214518.1,187.0,hypothetical protein Rv0004
4,4,Rv0005,mra:MRA_0005,A5TY73,chromosome,NC_000962.3,5240.0,7267.0,+,887081.0,gyrB,NP_214519.2,675.0,DNA gyrase subunit B


14- Fusion del dataframe de datos proteomicos con el del mapeo H37Ra a H37Rv

In [26]:
df.rename(columns = {'Majority protein IDs': 'Ra_UNIPROT'}, inplace = True)

In [27]:
merged_df = df.merge(df_rv_ra, how = 'outer', on = 'Ra_UNIPROT')


16- Redondeos de fold change y pvalue para mejor presentación

In [28]:
merged_df['fold_change'] = merged_df['fold_change'].round(decimals = 2)
merged_df['pvalue'] = merged_df['pvalue'].round(decimals = 4)

17- Eliminacion de locus no vistos o sin ortologos en ra 


In [29]:
merged_df = merged_df[merged_df['fold_change'].notnull()]

Muestra de como quedó el dataframe. Se muestran algunos ejemplos de proteínas cuyo p-value fue menor a 0.05

In [30]:
merged_df[['Locus tag', 'fold_change', 'pvalue','Protein Name', 'Locus' ]][merged_df['pvalue'] < 0.05].head(7)

,Locus tag,fold_change,pvalue,Protein Name,Locus
0,Rv0001,0.74,0.0061,chromosomal replication initiator protein DnaA,dnaA
1,Rv0002,0.68,0.0342,DNA polymerase III subunit beta,dnaN
2,Rv0003,0.67,0.0039,DNA replication/repair protein RecF,recF
16,Rv0021c,0.47,0.0309,hypothetical protein Rv0021c,NaN
29,Rv0046c,0.59,0.0106,inositol-3-phosphate synthase,ino1
30,Rv0047c,1.69,0.0118,hypothetical protein Rv0047c,NaN
36,Rv0054,1.21,0.0049,single-strand DNA-binding protein,ssb


18- Armado del volcano plot

In [31]:
import plotly.express as px
import math

fig = px.scatter(merged_df, x="log_fc", y="log_pvalue", hover_name="Locus tag", hover_data=["Locus", "Protein Name", "log_fc", 'pvalue', 'fold_change'])
fig.add_hline(y=-1.3)
fig.update_yaxes(autorange="reversed")



fig.show()

Herramientas para analizar alguna proteina particular por el Locus o Locus tag

In [32]:
merged_df.loc[merged_df['Locus'] == 'tgs2'][['Locus tag', 'fold_change', 'pvalue','Protein Name', 'Locus' ]]

,Locus tag,fold_change,pvalue,Protein Name,Locus
2087,Rv3734c,0.78,0.0739,diacyglycerol O-acyltransferase,tgs2


In [33]:
merged_df.loc[merged_df['Locus tag'] == 'Rv3130c'][['Locus tag', 'fold_change', 'pvalue','Protein Name', 'Locus' ]]

,Locus tag,fold_change,pvalue,Protein Name,Locus


In [34]:
sobreexp = merged_df[(merged_df["pvalue"] < 0.05)& (merged_df["fold_change"] > 1.3)].shape[0]
repr = merged_df[(merged_df["pvalue"] < 0.05)& (merged_df["fold_change"] < (1/1.3))].shape[0]
cambiadas = sobreexp + repr

print('Cantidad de proteínas sobreeexpresadas (fc > 1.3) = ', sobreexp)
print('Cantidad de proteínas reprimidas (fc < 1.3) = ', repr)
print('Total de proteínas expresadas diferencialmente = ', cambiadas)

Cantidad de proteínas sobreeexpresadas (fc > 1.3) =  76
Cantidad de proteínas reprimidas (fc < 1.3) =  201
Total de proteínas expresadas diferencialmente =  277


19 - Exportar el dataframe procesado. (Para descargar remover el # de la ultima fila)

In [35]:
simple_df_locus = merged_df[merged_df['Locus tag'].notna()][['Locus tag','Locus','log_fc','fold_change','pvalue','Ra_UNIPROT','Fasta headers']].sort_values(by=['log_fc'])

simple_df_locus[['Locus tag','Locus','log_fc','pvalue']].to_csv('simple_df_locus.csv', index=False)

from google.colab import files
# files.download("simple_df_locus.csv")

#Analisis sin la muestra Sin ATc - 2

20- Se crea una copia de log_muestras para no generar conflictos con el dataframe 1

In [36]:
log_muestras2 = log_muestras[:]
muestras2 = muestras[:]

21- Se elimina la muestra Sin ATc - 2 de la lista de muestra a procesar



In [37]:
log_muestras2.pop(1)
muestras2.pop(1)

'Sin ATc - 2'

In [38]:
muestras2

['Sin ATc - 1',
 'Sin ATc - 3',
 'Sin ATc - 5',
 '0.25 ATc - 1',
 '0.25 ATc - 3',
 '0.25 ATc - 4',
 '0.25 ATc - 5']

El resto del procesamiento es igual al del caso anterior con todas las muestras

In [39]:
df2['conteo_0'] = (df2[muestras2[:3]] != 0 ).sum(axis=1)
df2['conteo_0.25'] = (df2[muestras2[3:]] != 0 ).sum(axis=1)


df2 = df2[(df2['conteo_0'] >= 3) | (df2['conteo_0.25'] >= 3)]

In [40]:
print('Cantidad de filas restantes:', df2.shape[0])

Cantidad de filas restantes: 2171


In [41]:
def fill_na(x):
  if x == 0:
    a = np.nan
  else:
    a = math.log(x, 2)
    return a

In [42]:
log_muestras2 = []
for muestra in muestras2:
  nombre = 'log_' + muestra
  log_muestras2.append(nombre)
  df2[nombre] = df2[muestra].apply(fill_na)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
df2[log_muestras2] = padua.imputation.gaussian(df2[log_muestras2], width=0.3, downshift=-1.8, prefix=None)[0]

Una muestra de como queda formateado el dataframe

In [44]:
df2[log_muestras2].head(5)

,log_Sin ATc - 1,log_Sin ATc - 3,log_Sin ATc - 5,log_0.25 ATc - 1,log_0.25 ATc - 3,log_0.25 ATc - 4,log_0.25 ATc - 5
0,27.595838,27.777748,27.854955,27.409489,27.409003,27.464353,27.300424
1,29.931287,29.904692,30.127802,28.925356,29.666651,29.542354,29.794299
2,26.365246,26.213693,26.468865,25.503822,25.885625,25.904284,25.964970
3,29.538455,29.503606,29.578145,29.558774,29.478887,29.498327,28.890817
4,29.565327,29.616265,29.670134,29.473285,29.586545,29.496614,28.570873


22- Mismo t-test que en el caso anterior pero sin la muestra eliminada


In [45]:
df2['pvalue'] = scipy.stats.ttest_ind(df2[log_muestras2[:3]], df2[log_muestras2[3:]], axis=1)[1]

In [46]:
df2['log_pvalue'] = df2['pvalue'].apply(lambda x: math.log(x, 10))

In [47]:
df2[['pvalue','log_pvalue']].head(5)

,pvalue,log_pvalue
0,0.006035,-2.219338
1,0.084367,-1.073827
2,0.011995,-1.920997
3,0.368823,-0.433182
4,0.289228,-0.538760


In [48]:
for muestra in muestras2:
  nombre = 'log_' + muestra
  df2[muestra] =  2 ** df2[nombre]

In [49]:
df2['fold_change'] = df2[muestras2[3:]].mean(axis=1)/df2[muestras2[:3]].mean(axis=1)

In [50]:
df2['log_fc'] = df2['fold_change'].apply(lambda x: math.log(x, 2))

In [51]:
df2[['log_fc', 'log_pvalue']].head(5)

,log_fc,log_pvalue
0,-0.349855,-2.219338
1,-0.473493,-1.073827
2,-0.527434,-1.920997
3,-0.160112,-0.433182
4,-0.283445,-0.538760


In [52]:
df2.rename(columns = {'Majority protein IDs': 'Ra_UNIPROT'}, inplace = True)

In [53]:
merged_df2 = df2.merge(df_rv_ra, how = 'outer', on = 'Ra_UNIPROT')

In [54]:
## Redondeos 

merged_df2['fold_change'] = merged_df2['fold_change'].round(decimals = 2)
merged_df2['pvalue'] = merged_df2['pvalue'].round(decimals = 4)

## Eliminacion de locus no vistos o sin ortologos en ra 

merged_df2 = merged_df2[merged_df2['fold_change'].notnull()]

In [55]:
import plotly.express as px
import math

fig = px.scatter(merged_df2, x="log_fc", y="log_pvalue", hover_name="Locus tag", hover_data=["Locus", "Protein Name", "log_fc", 'pvalue', 'fold_change'])
fig.add_hline(y=-1.3)
fig.update_yaxes(autorange="reversed")



fig.show()

In [56]:
merged_df2.loc[merged_df2['Locus'] == 'fas'][['Locus tag', 'fold_change', 'pvalue','Protein Name', 'Locus' ]]

,Locus tag,fold_change,pvalue,Protein Name,Locus
1419,Rv2524c,0.49,0.0515,fatty acid synthase,fas


In [57]:
merged_df2.loc[merged_df2['Locus tag'] == 'Rv3392c'][['Locus tag', 'fold_change', 'pvalue','Protein Name', 'Locus' ]]

,Locus tag,fold_change,pvalue,Protein Name,Locus
1886,Rv3392c,0.79,0.0072,cyclopropane mycolic acid synthase CmaA,cmaA1


In [58]:
sobreexp = merged_df2[(merged_df2["pvalue"] < 0.05)& (merged_df2["fold_change"] > 1.3)].shape[0]
repr = merged_df2[(merged_df2["pvalue"] < 0.05)& (merged_df2["fold_change"] < (1/1.3))].shape[0]
cambiadas = sobreexp + repr

print('Cantidad de proteínas sobreeexpresadas (fc > 1.3) = ', sobreexp)
print('Cantidad de proteínas reprimidas (fc < 1.3) = ', repr)
print('Total de proteínas expresadas diferencialmente = ', cambiadas)

Cantidad de proteínas sobreeexpresadas (fc > 1.3) =  80
Cantidad de proteínas reprimidas (fc < 1.3) =  168
Total de proteínas expresadas diferencialmente =  248


Exportación. Remover # de la ultima fila para desacargar

In [59]:
simple_df2_locus = merged_df2[merged_df2['Locus tag'].notna()][['Locus tag','Locus','log_fc','fold_change','pvalue','Ra_UNIPROT','Fasta headers']].sort_values(by=['log_fc'])

simple_df2_locus[['Locus tag','Locus','log_fc','pvalue']].to_csv('simple_df2_locus.csv', index=False)

from google.colab import files
# files.download("simple_df2_locus.csv")

Comaparativa de los resultados en ambos procesamientos de algunas proteínas relevantes

In [60]:
prot_list = ['Rv1209','Rv1129c','Rv1131','Rv1130','Rv0467','Rv2930','Rv2941','Rv2940c','Rv0341',
             'Rv0343','Rv0342','Rv0971c','Rv1219c','Rv1218c','Rv3087','Rv1425','Rv1426c',
             'Rv1923','Rv0824c','Rv2247','Rv2482c' ,'Rv2187','Rv2244','Rv2245' ,'Rv2242','Rv2243',
             'Rv2246','Rv3281','Rv0824c','Rv1679','Rv0458' ,'Rv0761c','Rv1862','Rv2335','Rv3089',
             'Rv0166','Rv2524c','Rv0823c','Rv2187','Rv0971c','Rv0001','Rv2931','Rv2932','Rv2933',
             'Rv2231c','Rv3208','Rv1493','Rv0914c','Rv0974c','RV3736','Rv1267c']

In [61]:
for prot in prot_list:
  print('Locus tag:', prot)
  print('Todas las muestras')
  print(merged_df.loc[merged_df['Locus tag'] == prot][['fold_change', 'pvalue']])
  print('Sin muestra 2')
  print(merged_df2.loc[merged_df2['Locus tag'] == prot][['fold_change', 'pvalue']])

  print('\n')

Locus tag: Rv1209
Todas las muestras
     fold_change  pvalue
701         3.45  0.0095
Sin muestra 2
     fold_change  pvalue
695         3.05  0.0269


Locus tag: Rv1129c
Todas las muestras
     fold_change  pvalue
659         2.61  0.0007
Sin muestra 2
     fold_change  pvalue
653         2.65  0.0027


Locus tag: Rv1131
Todas las muestras
     fold_change  pvalue
661         6.52   0.019
Sin muestra 2
     fold_change  pvalue
655         5.25  0.0406


Locus tag: Rv1130
Todas las muestras
     fold_change  pvalue
660         6.52  0.0043
Sin muestra 2
     fold_change  pvalue
654         5.94  0.0136


Locus tag: Rv0467
Todas las muestras
     fold_change  pvalue
282         3.39  0.0188
Sin muestra 2
     fold_change  pvalue
279         3.53  0.0371


Locus tag: Rv2930
Todas las muestras
      fold_change  pvalue
1641         2.07  0.0276
Sin muestra 2
      fold_change  pvalue
1628         2.28   0.029


Locus tag: Rv2941
Todas las muestras
      fold_change  pvalue
1649         1